# ML Project 2: Segmentation of aerial images

Instructions: 
- exploratory data analysis to understand your dataset and your features
- feature processing and engineering to clean your dataset and extract more meaningful information
- implement and use machine learning methods on real data
- analyze your model and generate predictions using those methods and report your findings

**Submission deadline**. Dec 17th, 2020 (at 16:00 afternoon, sharp)

Deliverables at a glance. (More details and grading criteria further down)
- Written Report. You will write a maximum 4 page PDF report on your ndings, using LaTeX.
- Code. In Python. External libraries are allowed, if properly cited.
- Competitive Part. To give you immediate feedback and a fair ranking, we use the competition platform AIcrowd.com to score your predictions. You can submit whenever and almost as many times as you like, up until the final submission deadline.

**Goal**: For this problem, we provide a set of satellite/aerial images acquired from GoogleMaps. We also provide ground-truth images where each pixel is labeled as {road, background}. Your goal is to train a classifier to segment roads in these images, i.e. assign a label {road=1, background=0} to each pixel. Please see detailed instructions on the course github.

Good summary:https://neptune.ai/blog/image-segmentation-in-2020

### Imports

In [ ]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os, sys
from PIL import Image

import fastai
from fastai.vision import *
from fastai.callbacks import *
from fastai.utils.mem import *
import tensorflow as tf
from torchvision.models import vgg16_bn
import torchvision.transforms as transforms
from skimage import measure
from numpy import linalg as LA
import shutil

import warnings
warnings.filterwarnings('ignore') 

# helper functions from lab (load csv, create prediction, etc)
from helper_functions.helper_functions import *
# script to reconstruct an image from the sample submission file
from helper_functions.submission_to_mask import *

#script to make a submission file from a binary image:
from helper_functions.mask_to_submission import *

# Baseline for machine learning project on road segmentation.
from helper_functions.tf_aerial_images import *

# Baseline for machine learning project on road segmentation.
from helper_functions.helper_Marijn import *

%load_ext autoreload
%autoreload 2

Set CUDA if available

In [ ]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# Helper functions
def load_image(infilename):
    data = mpimg.imread(infilename)
    return data

def img_float_to_uint8(img):
    rimg = img - np.min(img)
    rimg = (rimg / np.max(rimg) * 255).round().astype(np.uint8)
    return rimg

# Concatenate an image and its groundtruth
def concatenate_images(img, gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = np.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = np.zeros((w, h, 3), dtype=np.uint8)
        gt_img8 = img_float_to_uint8(gt_img)          
        gt_img_3c[:,:,0] = gt_img8
        gt_img_3c[:,:,1] = gt_img8
        gt_img_3c[:,:,2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = np.concatenate((img8, gt_img_3c), axis=1)
    return cimg

def img_crop(im, w, h):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            if is_2d:
                im_patch = im[j:j+w, i:i+h]
            else:
                im_patch = im[j:j+w, i:i+h, :]
            list_patches.append(im_patch)
    return list_patches

## Data pre-processing: 

File descriptions:
- `training` - the training set consisting of images with their ground truth
- `test_set_images` - the test set
- `sampleSubmission.csv` - a sample submission file in the correct format
- `mask_to_submission.py` - script to make a submission file from a binary image
- `submission_to_mask.py` - script to reconstruct an image from the sample submission file
- `tf_aerial_images.py` - Baseline for machine learning project on road segmentation. This simple baseline consits of a CNN with two convolutional+pooling layers with a soft-max loss


The sample submission file contains two columns: 
- first column corresponds to the image id followed by the x and y top-left coordinate of the image patch (16x16 pixels)
- second column is the label assigned to the image patch

### Paths to data: 

In [ ]:
path = Path('../data/training/')
path_test = Path('../data/test_set_images/')

if not (path /'images').exists():
    (path /'images').mkdir()
    
if not (path / 'groundtruth').exists():
    (path / 'groundtruth').mkdir()

path_train = path /'images'
path_GT = path / 'groundtruth'

### Pixel range: 
Look at pixel values for GT and images. GT are supposed to be binary labels (0,1)

Get image file names:

In [ ]:
f_train_names = get_image_files(path_train)
f_gt_names = get_image_files(path_GT)

Look at pixel range for one GT and image:

In [ ]:
mask = open_mask(f_gt_names[0])
img = open_image(f_train_names[0])
img_gt = open_image(f_gt_names[0])

In [ ]:
fig, axs = plt.subplots(1,3, figsize = (5, 2))
axs[0].hist(img.data[0])
axs[0].set_title('Raw image')
axs[1].hist(mask.data[0])
axs[1].set_title('Mask')
axs[2].hist(img_gt.data[0])
axs[2].set_title('gt');

Comment: problem is that GT is not binary label, see some values between 0-1 will need to apply a threshold.

### Create patches: 
Create 16x16 patches of data and masks:

https://medium.com/analytics-vidhya/a-simple-cloud-detection-walk-through-using-convolutional-neural-network-cnn-and-u-net-and-bc745dda4b04

In [ ]:
f_train_names = get_image_files(path_train)
f_gt_names = get_image_files(path_GT)

In [ ]:
# Extract patches from input images
patch_size = 16  # each patch is 16*16 pixels

# Number of images to extract patches
N = 20
img_patches = [
    img_crop(mpimg.imread(img), patch_size, patch_size)
    for img in f_train_names[:N]
]
gt_patches = [
    img_crop(mpimg.imread(img), patch_size, patch_size) for img in f_gt_names[:N]
]

# Linearize list of patches
img_patches = np.asarray([
    img_patches[i][j] for i in range(len(img_patches))
    for j in range(len(img_patches[i]))
])
gt_patches = np.asarray([
    gt_patches[i][j] for i in range(len(gt_patches))
    for j in range(len(gt_patches[i]))
])
patch_shape = img_patches.shape[1]
print(f'Number of patches: {len(img_patches)} created from {N} images')
print(f'Shape of patches: {patch_shape}')

Create RGB PIL image from patches and save them. To meet the fastai requirements, we should organize our data into `patches/images` and `patches/labels`

In [ ]:
if not (path/'patches/images').exists():
    (path/'patches/images').mkdir()

if not (path/'patches/labels').exists():
    (path/'patches/labels').mkdir()
    
path_data = Path('../data/training/patches')
path_lbl = path_data/'labels'
path_img = path_data/'images'

Create RGB 16x16 patches of images and saves them:

In [ ]:
for i in range(len(img_patches)):
    rgb_patch = Image.fromarray((256 * img_patches[i]).astype(np.uint8), 'RGB')
    rgb_patch.save(path_img / f'patch_{i}.png')

#### Create binary masks: 

Convert the ground truth images to values 0 (no road) and 1 (road) and store them into a folder called `pacthes/labels`. In pillow, all images stored in 0-255 range so actually we give road the label (255). This will be corrected in the tf model later.

Show what this looks like for one mask:

In [ ]:
fig, ax = plt.subplots(1, 2)
im = open_image(f_gt_names[0])
print(f'Value range in GT:{np.unique(im.data.numpy())}')
# mask:
img = mpimg.imread(f_gt_names[0])
im_mask = Image.fromarray((np.where(img > 0.5, 1, 0) * 255).astype(np.uint8),
                          'L')
print(f'Value range in mask:{np.unique(im_mask)}')
im.show(ax[0])
ax[0].set_title('GT')
ax[1].imshow(np.asarray(im_mask), cmap='Greys_r')
ax[1].set_title('Binary mask');

Repeat for all masks:

In [ ]:
threshold = 0.5 
for i in range(len(gt_patches)):
    img = gt_patches[i]
    # Assign label "road" to all pixels above a threshold:
    im = Image.fromarray((np.where(img > threshold, 1, 0) * 255).astype(np.uint8))
    # save images:
    im.save(path_lbl / f'patch_{i}.png')

In [ ]:
# get images and labels filenames
img_names = get_image_files(path_img)
lbl_names = get_image_files(path_lbl)
print(len(img_names), len(lbl_names))

### Data bunch:

Load images and labels into tensors. Instead of using the entire set of images as training, we will let the library divide them into training (80%) and validation (20%) sets. Also, we will use some data augmentation. Data augmentation is a technique to increase the number of training samples by applying some random transformations like rotation, flipping, warp, and others.  Data Bunch keeps track of the samples and respective labels and, in the case of image segmentation, also merges both for a fast visualization

In [ ]:
# Classes for segmentation with 0,255 labels:
class SegLabelListCustom(SegmentationLabelList):
    def open(self, fn):
        return open_mask(fn, div=True)
class SegItemListCustom(SegmentationItemList):
    _label_cls = SegLabelListCustom

Data loader, normalize to imagenet statistics as unet pretrained on imagenet.

In [ ]:
bs = 4
print(f'Batch size:{bs}')
print(f'Patch shape:{patch_shape}')

src = (SegItemListCustom.from_folder(
    path_img).split_by_rand_pct().label_from_func(
        lambda x: path_lbl / x.relative_to(path_img), classes=['rest',
                                                               'road']))
data = (src.transform(get_transforms(flip_vert=True),
                      size=patch_shape,
                      tfm_y=True).databunch(bs=bs).normalize(imagenet_stats))
data

In [ ]:
data.show_batch(2)

Look at a few images with their masks: 

In [ ]:
fig, ax = plt.subplots(10,2, figsize=(10,6))

for i in range(10):
    im = data.valid_ds.x[i]
    mask = data.valid_ds.y[i]
    label = patch_to_label(mask.data.numpy())
    print(f'Label:{label}')
    ax[i,0].imshow(im.data.numpy()[0])
    ax[i,1].imshow(mask.data.numpy()[0], cmap='Greys_r')

## Training:

### Model:
pre-trained ResNet 34 version of the U-Net, that has 34 layers in the contracting path. To create it, we will define a accuracy function, to measure the performance of the mode, the weight decay (regularization to avoid overfitting of the model) value and the learning rate (rate that will be multiplied to the gradient to adjust parameters during back-propagation step).

accuracy in an image segmentation problem is the same as that in any classification problem. `Accuracy = no. of correctly classified pixels / total no. of pixels`

In [ ]:
def acc_metric(input, target):
    target = target.squeeze(1)
    return (input.argmax(dim=1)==target).float().mean()

wd = 1e-2

lr=1e-3

learn = unet_learner(data, models.resnet34, metrics=acc_metric, wd=wd)

In [ ]:
lr_find(learn)
learn.recorder.plot()

### First cycle:

In [ ]:
sim = 2

In [ ]:
lr = 1e-3

In [ ]:
def do_fit(save_name, lrs=lr, pct_start=0.9):
    """
    do_fit: fits during 3 epochs with feature loss. 
    """
    learn.fit_one_cycle(3, lrs, pct_start=pct_start)
    learn.save(save_name, return_path=True)
    learn.show_results(rows=1, imgsize=10)
    learn.recorder.plot_losses()
    learn.recorder.plot_metrics()

In [ ]:
do_fit('sim{}_1a'.format(sim), slice(1e-6, 1e-3))

In [ ]:
img = learn.data.valid_ds.x[22]
mask = learn.data.valid_ds.y[22]
pred = learn.predict(img)[0]
fig, ax = plt.subplots(1, 3, figsize=(12, 6))
img.show(ax[0])
mask.show(ax[1])
pred.show(ax[2])

In [ ]:
fig, ax = plt.subplots(20, 3, figsize=(10, 20))
for i in range(20):
    img = learn.data.valid_ds.x[i]
    mask = learn.data.valid_ds.y[i]
    pred = learn.predict(img)[0]
    img.show(ax[i, 0])
    mask.show(ax[i, 1])
    pred.show(ax[i, 2])

Problem: not thaaat great but better...

## Create predictions :
Your predictions must be in .csv format, see sample-submission.csv. You must use the same datapoint ids as in the test set test.csv. To generate .csv output from Python, use our provided helper functions in helpers.py (see Project 1 folder on github).

In [ ]:
if not (Path('../data/test_set_images/all_tests')).exists():
    (Path('../data/test_set_images/all_tests')).mkdir()
    
if not (Path('../data/test_set_images/all_tests/patches')).exists():
    (Path('../data/test_set_images/all_tests/patches')).mkdir()

In [ ]:
! find ../data/test_images -type f -name "*.png" -exec mv "{}" ../data/test_images/all_tests \;

In [ ]:
path_test = Path('../data/test_set_images/all_tests')
fnames_test = get_image_files(path_test)

path_test_patches = Path('../data/test_set_images/all_tests/patches')

### Pre-process test set:

Create the list of submission names (for each test patch of 16x16 top left coordinate):

In [ ]:
fnames_test_patches = []
test_img = open_image(fnames_test[0])
for img_number in range(1,51):
    for j in range(0, test_img.shape[1], patch_size):
            for i in range(0,  test_img.shape[2], patch_size):
                name = "{:03d}_{}_{}.png".format(img_number, j, i)
                fnames_test_patches.append(name)
len(fnames_test_patches)

Create test patches and save them:

In [ ]:
def create_test_patches(test_file_name):
    img_number = int(re.search(r"\d+", str(test_file_name)).group(0))
    im = mpimg.imread(Path(test_file_name))
    patch_size = 16
    for j in range(0, im.shape[1], patch_size):
        for i in range(0, im.shape[0], patch_size):
            patch = im[i:i + patch_size, j:j + patch_size]
            rgb_patch = Image.fromarray((256*patch).astype(np.uint8), 'RGB')
            name = "{:03d}_{}_{}.png".format(img_number, j, i)
            rgb_patch.save(path_test_patches/name)

In [ ]:
for test_im in fnames_test: 
    create_test_patches(test_im)

In [ ]:
print(f'Number of created test patches: {len(get_image_files(path_test_patches))}')

In [ ]:
"""Reads a single image and outputs the strings that should go into the submission file"""
def label_predictions(test_file_name, path_test_patches):
    # prediction from patch: 
    tensor = open_image(path_test_patches/name)
    prediction = learn.predict(tensor)
    label = patch_to_label(prediction[0].data.numpy())
    yield("{},{}".format(test_file_name, label))

In [ ]:
from tqdm import tqdm
def masks_to_submission(submission_filename, image_filenames, path_test_patches):
    """Converts images into a submission file"""
    with open(submission_filename, 'w') as f:
        f.write('id,prediction\n')
        for i in tqdm(range8len(image_filenames)):
            fn = image_filenames[i]
            f.writelines('{}\n'.format(s) for s in label_predictions(fn, path_test_patches))

In [ ]:
masks_to_submission('dummy_submission.csv', fnames_test_patches, path_test_patches)